# Cloud Workshop Azure Databricks
## 08. Cross Validation

## Using Cross Validation

In this exercise, you will use cross-validation to optimize parameters for a regression model.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

# Load the source data
# File location and type
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

data = data.select("DayofMonth", "DayOfWeek", "Carrier", "OriginAirportID", "DestAirportID", "DepDelay", col("ArrDelay").alias("label"))

# Split the data
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]

### Define the Pipeline
Now define a pipeline that creates a feature vector and trains a regression model

In [5]:
monthdayIndexer = StringIndexer(inputCol="DayofMonth", outputCol="DayofMonthIdx")
weekdayIndexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeekIdx")
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
originIndexer = StringIndexer(inputCol="OriginAirportID", outputCol="OriginAirportIdx")
destIndexer = StringIndexer(inputCol="DestAirportID", outputCol="DestAirportIdx")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normNums")

featVect = VectorAssembler(inputCols=["DayofMonthIdx", "DayOfWeekIdx", "CarrierIdx", "OriginAirportIdx", "DestAirportIdx", "normNums"], outputCol="features")

lr = LinearRegression(labelCol="label", featuresCol="features")

pipeline = Pipeline(stages=[monthdayIndexer, weekdayIndexer, carrierIndexer, originIndexer, destIndexer, numVect, minMax, featVect, lr])

### Tune Parameters
You can tune parameters to find the best model for your data. To do this you can use the  **CrossValidator** class to evaluate each combination of parameters defined in a **ParameterGrid** against multiple *folds* of the data split into training and validation datasets, in order to find the best performing parameters. Note that this can take a long time to run because every parameter combination is tried multiple times.

In [7]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.01]).addGrid(lr.maxIter, [10, 5]).build()
cv = CrossValidator(estimator=pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, numFolds=2)

model = cv.fit(train)

### Test the Model
Now you're ready to apply the model to the test data.

In [9]:
prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "label")

predicted.show()

+--------------------+--------------------+-----+
 features| prediction|label|
+--------------------+--------------------+-----+
[25.0,2.0,10.0,1....| 91.58473682979744| 113|
[25.0,2.0,10.0,1....| 3.673759614867329| 2|
[25.0,2.0,10.0,57...| 20.44984362421652| 41|
[25.0,2.0,10.0,49...| 15.050521943783146| 7|
[25.0,2.0,10.0,49...| 93.23123917855565| 82|
[25.0,2.0,10.0,38...| 61.0453893673748| 56|
[25.0,2.0,10.0,38...| 5.862626215107419| 21|
[25.0,2.0,10.0,46...| -3.037552717914437| 28|
[25.0,2.0,10.0,12...| -1.294041519181313| -20|
[25.0,2.0,10.0,12...| 30.368569231029205| 0|
[25.0,2.0,10.0,12...| 44.51366275707924| 57|
[25.0,2.0,10.0,12...| 124.4027919099164| 103|
[25.0,2.0,10.0,12...| 2.877584663268266| 67|
[25.0,2.0,10.0,48...| -3.8263847273858502| -8|
[25.0,2.0,10.0,48...| -2.3399734994154215| 38|
[25.0,2.0,10.0,0....| 42.7980314645014| 32|
[25.0,2.0,10.0,64...| -7.836850171530266| -25|
[25.0,2.0,10.0,64...| 48.743523932669895| 38|
[25.0,2.0,2.0,1.0...| 35.328192931242874| 30|
[25.0,2.0,2.0,27....|0.028373414032834887| 26|
+--------------------+--------------------+-----+
only showing top 20 rows

### Examine the Predicted and Actual Values
You can plot the predicted values against the actual values to see how accurately the model has predicted. In a perfect model, the resulting scatter plot should form a perfect diagonal line with each predicted value being identical to the actual value - in practice, some variance is to be expected.
Run the cells below to create a temporary table from the **predicted** DataFrame and then retrieve the predicted and actual label values using SQL. You can then display the results as a scatter plot, specifying **-** as the function to show the unaggregated values.

In [11]:
predicted.createOrReplaceTempView("regressionPredictions")

In [12]:
%sql
SELECT label, prediction FROM regressionPredictions

label,prediction
113,91.58473682979744
2,3.673759614867329
41,20.44984362421652
7,15.050521943783146
82,93.23123917855565
56,61.0453893673748
21,5.862626215107419
28,-3.037552717914437
-20,-1.294041519181313
0,30.368569231029205


### Retrieve the Root Mean Square Error (RMSE)
There are a number of metrics used to measure the variance between predicted and actual values. Of these, the root mean square error (RMSE) is a commonly used value that is measured in the same units as the prediced and actual values - so in this case, the RMSE indicates the average number of minutes between predicted and actual flight delay values. You can use the **RegressionEvaluator** class to retrieve the RMSE.

In [14]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 17.30518072316676

> Fin